In [1]:
import abc

import numpy as np

class Graph(abc.ABC):
    def __init__(self, numVertices, directed=False):
        self.numVertices = numVertices
        self.directed = directed
        
    @abc.abstractmethod
    def add_edge(self, v1, v2, weight):
        pass
    
    @abc.abstractmethod
    def get_adjacent_vertices(self, v):
        pass
    
    @abc.abstractmethod
    def get_indegree(self, v):
        pass
    
    @abc.abstractmethod
    def get_edge_weight(self, v1, v2):
        pass
    
    @abc.abstractmethod
    def display(self):
        pass

In [2]:
class AdjacencyMatrixGraph(Graph):
    def __init__(self, numVertices, directed=False):
        super(AdjacencyMatrixGraph, self).__init__(numVertices, directed)
        
        self.matrix = np.zeros((numVertices, numVertices))
        
        
    def add_edge(self, v1, v2, weight=1):
        if v1 >= self.numVertices or v2 >= self.numVertices or v1 < 0 or v2 < 0:
            raise ValueError("Vertices %d and %d are out of bounds" % (v1, v2))
            
        if weight < 1:
            raise ValueError("An edge cannot have weight < 1")
            
        self.matrix[v1][v2] = weight
        
        if self.directed == False:
            self.matrix[v2][v1] = weight
            
            
    def get_adjacent_vertices(self, v):
        if v < 0 or v >= self.numVertices:
            raise ValueError("Cannot access vertex %d" % v)
            
        adjacent_vertices = []
        for i in range(self.numVertices):
            if self.matrix[v][i] > 0:
                adjacent_vertices.append(i)
                
        return adjacent_vertices
    
    
    def get_indegree(self, v):
        if v < 0 or v >= self.numVertices:
            raise ValueError("Cannot access vertex %d" % v)
            
        indegree = 0
        for i in range(self.numVertices):
            if self.matrix[i][v] > 0:
                indegree = indegree + 1
                
        return indegree
    
    
    def get_edge_weight(self, v1, v2):
        return self.matrix[v1][v2]
    
    
    def display(self):
        for i in range(self.numVertices):
            for v in self.get_adjacent_vertices(i):
                print(i, "-->", v)

In [3]:
class Node:
    def __init__(self, vertexId):
        self.vertexId = vertexId
        self.adjacency_set = set()
        
    def add_edge(self, v):
        if self.vertexId == v:
            raise ValueError("The vertex %d cannot be adjacent to itself" % v)
            
        self.adjacency_set.add(v)
        
    def get_adjacent_vertices(self):
        return sorted(self.adjacency_set)

In [4]:
class AdjacencySetGraph(Graph):
    def __init__(self, numVertices, directed=False):
        super(AdjacencySetGraph, self).__init__(numVertices, directed)
        
        self.vertex_list = []
        for i in range(numVertices):
            self.vertex_list.append(Node(i))
            
            
    def add_edge(self, v1, v2, weight=1):
        if v1 >= self.numVertices or v2 >= self.numVertices or v1 < 0 or v2 < 0:
            raise ValueError("Vertices %d and %d are out of bounds" % (v1, v2))
            
        if weight != 1:
            raise ValueError("An adjacency set cannot represent edge weights > 1")
            
        self.vertex_list[v1].add_edge(v2)
        
        if self.directed == False:
            self.vertex_list[v2].add_edge(v1)


    def get_adjacent_vertices(self, v):
        if v < 0 or v >= self.numVertices:
            raise ValueError("Cannot access vertex %d" % v)
            
        return self.vertex_list[v].get_adjacent_vertices()


    def get_indegree(self, v):
        if v < 0 or v >= self.numVertices:
            raise ValueError("Cannot access vertex %d" % v)
            
        indegree = 0
        for i in range(self.numVertices):
            if v in self.get_adjacent_vertices(i):
                indegree = indegree + 1
                
        return indegree


    def get_edge_weight(self, v1, v2):
        return 1
    
    
    def display(self):
        for i in range(self.numVertices):
            for v in self.get_adjacent_vertices(i):
                print(i, "-->", v)

In [7]:
from queue import Queue

def topological_sort(graph):
    queue = Queue()
    indegreeMap = {}
    
    for i in range(graph.numVertices):
        indegreeMap[i] = graph.get_indegree(i)
        if indegreeMap[i] == 0:
            queue.put(i)
            
    sortedList = []
    while queue:
        vertex = queue.get()
        sortedList.append(vertex)
        for v in graph.get_adjacent_vertices(vertex):
            indegreeMap[v] = indegreeMap[v] - 1
            if indegreeMap[v] == 0:
                queue.put(v)
            
    if len(sortedList) != graph.numVertices:
        raise ValueError("This graph has a cycle")
    
    print(sortedList)

In [9]:
from queue import Queue

def build_distance_table(graph, source):
    # vertex -> (distance from source, last vertex on path from source)
    distance_table = {}
    for vertex in range(graph.numVertices):
        distance_table[vertex] = (None, None)
    
    # the distance to the source from itself is 0
    distance_table[source] = (0, source)
    
    queue = Queue()
    queue.put(source)
    while not queue.empty():
        current_vertex = queue.get()
        current_distance = distance_table[current_vertex][0]
        for neighbor in graph.get_adjacent_vertices(current_vertex):
            # process only the node was not visited
            if distance_table[neighbor][0] is None:
                distance_table[neighbor] = (1 + current_distance, current_vertex)
                
                # enqueue neighbor if it has other adjacent nodes to explore
                if len(graph.get_adjacent_vertices(neighbor)) > 0:
                    queue.put(neighbor)
                    
    return distance_table

def shortest_path(graph, source, destination):
    distance_table = build_distance_table(graph, source)
    
    path = [destination]
    
    previous_vertex = distance_table[destination][1]
    while previous_vertex is not None and previous_vertex is not source:
        path = [previous_vertex] + path # Interesting way to add to the beginning of list
        previous_vertex = distance_table[previous_vertex][1]
        
    if previous_vertex is None:
        print(f"There is no path from {source} to {destination}")
    else:
        path = [source] + path
        print(f"Shortest path is {path}")
